<a href="https://colab.research.google.com/github/dhruvipatel14/Fashion_image_caption_prediction/blob/main/Text_mercato_Fashion_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', -1) 
import string

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/Text_mercato_fashion_data/train.csv')
train_df = pd.DataFrame(train_data)
test_data = pd.read_csv('/content/drive/MyDrive/Text_mercato_fashion_data/test.csv')
test_df = pd.DataFrame(test_data)
train_df.head()

,Title,Description,Material,Pattern,Neckline,Image_Path
0,Peach Poly Crepe jumpsuit,"This stylish foil print kurta from janasya is made of poly crepe and comes in an attractive peach color. It features 3/4 sleeve,round neck,a-line and it is calf length kurta that is suitable for casual occasions. Team it with matching leggings for a chic look.",Crepe,Printed,Round Neck,/images/pic_0.jpg
1,Light Brown Bias Yoke Checks Top,"This check pattern top by Work Label is crafted in cotton. Featuring a bias check at the yoke and straight check pattern in bottom half, a smart round Neckline, 3/4th sleeves, this mid hip length top offers a stylish & comfortable fit. Style this top with a trouser/skirt and medium high heels for chic look at work. This top can also be styled with a pair of stud earrings and a pair your regular sneakers to attain a casual look.",Cotton,Checks,Round Neck,/images/pic_1.jpg
2,Off White Geometric Straight Cotton Dobby Top With Skirt (Set of 2),"Featuring elegant printed details, this off white top and skirt set from Jaipur Kurti makes a statement addition to your casual wardrobe. Style this set with a pair of high heels and statement accessories to complete the look.",Viscose,Checks,Round Neck,/images/pic_2.jpg
3,Blue Me Away Cape Top,"Add an extra dose of style to your casual wardrobe with this elegant blue cape top from Twenty Dresses. Style With: A pair of black denims, colour block heels and statement earrings will complete this look!",Polyester,Solid/Plain,V-Neck,/images/pic_3.jpg
4,Yellow On A High Gown,Yellow polyester georgette maxi dress. Polyester knit lining inside for comfort. Overlapping v neckline with gathers at bust. Flared sleeves with attached belt at the waist. Concealed zipper on the left.,Polyester,Solid/Plain,V-Neck,/images/pic_4.jpg


Extracting features from Image dataset using VGG

In [4]:
# !unzip /content/drive/MyDrive/Text_mercato_fashion_data/train_image.zip -d /content/drive/MyDrive/Text_mercato_fashion_data/train_image

In [5]:
# !unzip /content/drive/MyDrive/Text_mercato_fashion_data/test_data.zip -d /content/drive/MyDrive/Text_mercato_fashion_data/

In [6]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model

In [7]:
def extract_features(directory):
  model = VGG16()
  model = Model(inputs= model.inputs,outputs = model.layers[-2].output)
  print(model.summary())

  features = dict()
  for name in listdir(directory):
    filename = directory + '/' + name
    img = load_img(filename,target_size=(224,224))
    img = img_to_array(img)
    img = img.reshape((1,img.shape[0],img.shape[1],img.shape[2]))
    img = preprocess_input(img)
    feature = model.predict(img,verbose = 0)
    img_id = name.split('.')[0]
    features[img_id] = feature
  return features

In [8]:
# extract features from all images
train_dict = '/content/drive/MyDrive/Text_mercato_fashion_data/train_image/train_image'
test_dict = '/content/drive/MyDrive/Text_mercato_fashion_data/test_data'
train_features = extract_features(train_dict)
test_features = extract_features(test_dict)
print('Extracted Train Features: %d' % len(train_features))
print('Extracted Dev Features: %d' % len(test_features))
# save to file
dump(train_features, open('train_features.pkl', 'wb'))
dump(test_features,open('test_features.pkl','wb'))

553467904/553467096 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

Preparing Text data : Extracting and cleaning image discription from dataset

In [9]:
# def clean_text(desc):
#   table = str.maketrans('', '', string.punctuation)
#   desc = desc.split()
#   desc = [word.lower() for word in desc]
#   desc = [w.translate(table) for w in desc]
#   desc = [word for word in desc if len(word)>1]
#   desc = [word for word in desc if word.isalpha()]
#   return desc

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords 
import re
stopword = set(stopwords.words('english') + list(punctuation))

In [12]:
# def tokens(text):
#   "List all the word tokens in a text."
#   text = re.sub('[0-9]+', '', text)
#   ptrn = re.compile('<.*?==>')
#   text = re.sub(ptrn, '', text)
#   text = text.lower()
#   return text

In [13]:
# train_df['Description'] = train_df['Description'].apply(lambda x : tokens(x))
# train_df['Description'] = train_df['Description'].apply(lambda x : word_tokenize(x))
# train_df['Description'] = train_df['Description'].apply(lambda x : [item for item in x if item not in stopword])
# train_df['Image_Path']= train_df['Image_Path'].apply(lambda s : s.split('/')[-1].split('.')[0])
# # train_df.set_index(train_df['Image_Path'],inplace= True)
# # train_df.drop('Image_Path',axis=1,inplace=True)
# # description_mapping = train_df.to_dict('index')
# # description_mapping



In [14]:
# train_df.set_index(train_df['Image_Path'],inplace= True)
# train_df.drop('Image_Path',axis=1,inplace=True)
# description_mapping = train_df.to_dict('index')
# description_mapping

In [15]:
# d = train_df.set_index('Image_Path')['Description'].to_dict()
# d

In [16]:
# train_df

In [17]:
# train_df['Description'] = train_df['Description'].apply(lambda x : clean_text(x))
# train_df['Description']

In [18]:
def tokens(text):
  "List all the word tokens in a text."
  text = re.sub('[0-9]+', '', text)
  ptrn = re.compile('<.*?==>')
  text = re.sub(ptrn, '', text)
  text = text.lower()
  return text

def description_map(df):
  desc_df = df[['Description','Image_Path']]
  desc_df['Image_Path']= desc_df['Image_Path'].apply(lambda s : s.split('/')[-1].split('.')[0])
  desc_df['Description'] = desc_df['Description'].apply(lambda x : tokens(x))
  desc_df['Description'] = desc_df['Description'].apply(lambda x : word_tokenize(x))
  desc_df['Description'] = desc_df['Description'].apply(lambda x : [item for item in x if item not in stopword])
  # desc_df.set_index(desc_df['Image_Path'],inplace= True)
  # desc_df.drop('Image_Path',axis=1,inplace=True)
  # description_mapping = desc_df.to_dict()
  description_mapping = desc_df.set_index('Image_Path')['Description'].to_dict()
  return description_mapping

def to_vocabulary(descriptions):
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc
 
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [19]:
training_mapping = description_map(train_df)
testing_mapping = description_map(test_df)

train_vocabulary = to_vocabulary(training_mapping)
print('Vocabulary Size: %d' % len(train_vocabulary))
# save to file
save_descriptions(training_mapping, 'train_descriptions.txt')
save_descriptions(testing_mapping,'test_descriptions.txt')

Vocabulary Size: 1365


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Preparing data for model training
text data and image data

In [20]:
from numpy import array,argmax
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, plot_model
from keras.models import Model
from keras.layers import Input,Dense,LSTM,Embedding,Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [21]:
# desc = dict()
# for key,val in training_mapping.items():
#   if key not in desc:
#     desc[key] = list()
#   seq = 'startseq ' + ' '.join(val) + ' endseq'
#   desc[key].append(seq)
# print(desc)

In [22]:
def load_clean_descriptions(mapping):
  desc = dict()
  for key,val in mapping.items():
    if key not in desc:
      desc[key] = list()
    seq = 'startseq ' + ' '.join(val) + ' endseq'
    desc[key].append(seq)
  return desc

# load photo features
def load_photo_features(filename):
	features = pickle.load(open(filename, 'rb'))
	return features

def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each image identifier
	for key, desc_list in descriptions.items():
		# walk through each description for the image
		for desc in desc_list:
			# encode the sequence
			seq = tokenizer.texts_to_sequences([desc])[0]
			# split one sequence into multiple X,y pairs
			for i in range(1, len(seq)):
				# split into input and output pair
				in_seq, out_seq = seq[:i], seq[i]
				# pad input sequence
				in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
				# encode output sequence
				out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
				# store
				X1.append(photos[key][0])
				X2.append(in_seq)
				y.append(out_seq)
	return array(X1), array(X2), array(y)

# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(256)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	# compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	model.summary()
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
	# loop for ever over images
	while 1:
		for key, desc_list in descriptions.items():
			# retrieve the photo feature
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
			yield [in_img, in_seq], out_word

Training sequences



In [23]:
train_descriptions = load_clean_descriptions(training_mapping)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('train_features.pkl')
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)

Descriptions: train=400
Photos: train=400
Vocabulary Size: 1305


Dev Seqences

In [24]:
# dev dataset
 
test_descriptions = load_clean_descriptions(testing_mapping)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('test_features.pkl')
print('Photos: test=%d' % len(test_features))
# prepare sequences
X1test, X2test, ytest = create_sequences(tokenizer, max_length, test_descriptions, test_features, vocab_size)

Descriptions: test=100
Photos: test=100


Model Training

In [25]:
# define the model
model = define_model(vocab_size, max_length)
# define checkpoint callback
filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
model.fit([X1train, X2train], ytrain, epochs=20, verbose=2, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 89)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 89, 256)      334080      input_4[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 4096)         0           input_3[0][0]                    
____________________________________________________________________________________________

Model Evaluation

In [26]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
  in_text = 'startseq'
  for i in range(max_length):
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    sequence = pad_sequences([sequence],maxlen = max_length)
    yhat = model.predict([photo,sequence],verbose=0)
    yhat = argmax(yhat)
    word = word_for_id(yhat,tokenizer)
    if word is None:
      break
      in_text += ' ' + word
      if word == 'endseq':
        break
  return in_text


# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [27]:
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

BLEU-1: 0.000000
BLEU-2: 0.000000
BLEU-3: 0.000000
BLEU-4: 0.000000


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [28]:
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Generating caption on unseen image

In [35]:
from pickle import load
from numpy import argmax
from keras.preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model

# extract features from each photo in the directory
def extract_features(filename):
  model = VGG16()
  model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
  image = load_img(filename,target_size=(224,224))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  feature = model.predict(image, verbose=0)
  return feature

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	in_text = 'startseq'
	for i in range(max_length):
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		sequence = pad_sequences([sequence], maxlen=max_length)
		yhat = model.predict([photo,sequence], verbose=0)
		yhat = argmax(yhat)
		word = word_for_id(yhat, tokenizer)
		if word is None:
			break
		in_text += ' ' + word
		if word == 'endseq':
			break
	return in_text

# load the tokenizer
tokenizer = load(open('/content/tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 89
# load the model
model = load_model('/content/model-ep005-loss3.696-val_loss4.529.h5')
# load and prepare the photograph
photo = extract_features('images.jpg')
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

startseq black polyester crepe jumpsuit sleeveless v neckline elasticated back zipper zipper fastening ease endseq
